# Getting started example

This is a first example of a Python's script using Vortex :

* It doesn't use the Vortex job management system (it would be to complex for a first example)
* The "Getting Started" page is not enough to understand every bit of this script: try not to focus to much on the details but instead look for the general concepts listed in the "Getting Started" page
* It should run on any workstation with Vortex installed and a configured FTP access to Meteo-France's mass archive (.netrc file with a ``hendrix.meteo.fr`` entry)
* For clarity sake, this example is presented in an IPython's notebook. However, it should also run in a raw Python's script

## Loading Vortex

In [1]:
from bronx.stdtypes import date
import vortex

# Load additional packages needed in this example
import common, sandbox
# Import the toolbox module that contains very usefull shortcut to use Vortex painlessly
from vortex import toolbox

# Get the session's ticket and shell objects for latter use:
t = vortex.ticket()
sh = t.sh
print "session's infos:"
print  t.idcard()

# Let's decrease the verbosity
t.warning()

# Just go to a cleaned temporary directory
tmpdir = sh.path.join(t.env.HOME, 'tmp', 'getting_started_demo')
sh.rm(tmpdir)
sh.cd(tmpdir, create=True)

Vortex 1.4.0 loaded ( Thursday 09. August 2018, at 18:05:53 )
session's infos:
+ Name     = root
+ Started  = 2018-08-09T18:05:53.105047Z
+ Opened   = True
+ Duration = PT0S
+ Loglevel = INFO


True

## Set some defaults because laziness is good...

In [2]:
# We will work on Arpege, PEARP data :
# this is a global configuration that should be added to the glove
t.glove.vapp = 'arpege'
t.glove.vconf = 'pearp'
# In this whole script, we will work on the 01/06/2017 18UTC run (production cutoff) :
# when looking for data, we will use those defaults
vortex.setup.defaults.update(
    date = date.Date('2017060118'),
    cutoff = 'production',
    model = 'arpege'
)
# Because we will send an e-mail later on...
vortex.setup.defaults.update(
    smtpserver='smtp.meteo.fr',
)

Note: Vortex provides a wide range of utility classes such as Date... 

## Get data from the mass archive

In [3]:
# We are not creating the resource, Provider, Container and Handler objects manually (it would be quite long).
# Instead, we are using the toolbox module...
rhandlers = toolbox.input(role='Gridpoint',
                          # The Resource part...
                          kind='gridpoint',
                          geometry='glob05',
                          nativefmt='grib',
                          origin='historic',
                          term='0,6',
                          # The provider part...
                          namespace='vortex.archive.fr',
                          block='forecast',
                          member='range(0,2)', # Only members 0, 1 and 2 (faster)
                          experiment='dble',   # The e-suite'
                          # The container part
                          filename='grib_glob05_m[member]_[term:fmth]'
                         )

# New input section with options :
+ role         = Gridpoint
# Resource handler description :
+ block        = forecast
+ experiment   = dble
+ filename     = grib_glob05_m[member]_[term:fmth]
+ geometry     = glob05
+ insitu       = False
+ kind         = gridpoint
+ member       = range(0,2)
+ namespace    = vortex.archive.fr
+ nativefmt    = grib
+ origin       = historic
+ term         = 0,6
# This command options :
+ complete     = False
+ loglevel     = None
+ now          = False
+ verbose      = True


At this point the resource Handlers objects are created but the data are still not downloaded :

In [4]:
print 'There are {:d} resource Handlers.'.format(len(rhandlers))

There are 6 resource Handlers.


Where are located the remote data (for the first remote handlers) ?

In [5]:
print rhandlers[0].location()

vortex://vsop.archive.fr/arpege/pearp/DBLE/20170601T1800P/mb000/forecast/grid.arpege-forecast.glob05+0000:00.grib


Let's download the files...

In [6]:
for rhandler in rhandlers:
    print rhandler.container.filename, ":", rhandler.get()

grib_glob05_m000_0000 : True
grib_glob05_m001_0000 : True
grib_glob05_m002_0000 : True
grib_glob05_m000_0006 : True
grib_glob05_m001_0006 : True
grib_glob05_m002_0006 : True


## Using the current active context

The current active Context, has recorded each of the **get** method calls. To illustrate that, it's able to generate a summary of all the input files:

In [7]:
report = t.context.sequence.inputs_report()
report.print_report()

* present                   : grib_glob05_m000_0000
* present                   : grib_glob05_m000_0006
* present                   : grib_glob05_m001_0000
* present                   : grib_glob05_m001_0006
* present                   : grib_glob05_m002_0000
* present                   : grib_glob05_m002_0006



## Using an Algo Component

We are using a very simple AlgoComponent specialy created for demonstration purposes: It looks for available GRIB files in the active Context's sequence, compute their size and MD5 sum ; finaly it saves the result in a json file. Of course, in real life, we usually want to run some kind of program or script.

Let's create the AlgoComponent object and run this wonder...

In [8]:
algo = toolbox.algo(kind='gribinfos', jsonoutput='super.json')
algo.run()
print "What's the result ?"
with open("super.json", 'r') as jsonfh:
    print ''.join(jsonfh.readlines())

# Loading algo component with description: :
+ jsonoutput   = super.json
+ kind         = gribinfos

----------------------------------------------------------------------------------------------------

01. <sandbox.algo.stdpost.GribInfos object at 0x7f4034d92950>
  kind: gribinfos
  engine: algo

 ----------------------------------------------------------------------------------------------------
# gribinfos : directory listing (post-run)                                                         #
----------------------------------------------------------------------------------------------------
What's the result ?
[
  {
    "vapp": "arpege", 
    "vconf": "pearp", 
    "member": 000, 
    "domain": "GLOB05", 
    "terms": {
      "0000:00": {
        "md5sum": "add8b43ac4359066f766fb459747f3a0", 
        "filesize": 18039405
      }, 
      "0006:00": {
        "md5sum": "407907be9cdf1566ac4a269fcd8e4d26", 
        "filesize": 17671950
      }
    }
  }, 
  {
    "vapp": "arpege", 
  

## Archive the output JSON file to the mass archive

For this demonstration, we will use a fake experiment ID called "demo"... (Note: a dedicated Resource class has been created for this demonstration purpose).

In [9]:
rhandlers = toolbox.output(# The Resource part...
                           kind='gribinfos',
                           # The provider part...
                           namespace='vortex.archive.fr',
                           block='forecast',
                           experiment='demo@meunierlf',
                           # The container part
                           filename='super.json'
                          )

# New output section with options :
# Resource handler description :
+ block        = forecast
+ experiment   = demo@meunierlf
+ filename     = super.json
+ kind         = gribinfos
+ namespace    = vortex.archive.fr
# This command options :
+ complete     = False
+ loglevel     = None
+ now          = False
+ verbose      = True


In [10]:
for rhandler in rhandlers:
    print rhandler.container.filename, '...'
    print 'to:', rhandler.location()
    rhandler.put()

super.json ...
to: vortex://vortex-free.archive.fr/arpege/pearp/demo@meunierlf/20170601T1800P/forecast/gribinfos.arpege.json


# [2018/08/09-18:06:04][vortex.tools.net][put:0311][WARNING]: Replacing <file:~meunierlf/vortex/arpege/pearp/demo/20170601T1800P/forecast/gribinfos.arpege.json>


## Some like SPAM

Just send the output file by email:

In [11]:
from vortex.tools.actions import actiond as ad
ad.mail(to='louis-francois.meunier@meteo.fr', 
        subject="PEARP's GRIB info file on {.resource.date!s}".format(rhandlers[0]),
        message='Hi,\n\nPlease have a look at the attached file.',
        attachments=(rhandlers[0].container.localpath(), )
       )

[2035]

Note: The Resource object attribute "date" is used to print the date in the email subject, while the Container's localpath method returns the path to the local file.